In [2]:
import numpy as np
import random

# Maze layout represented as a 2D NumPy array
# 0 = Free space, -1 = Obstacle, 1 = Goal (target for the agent)
maze = np.array([
    [0, 0, 0, -1, 0],
    [0, -1, 0, -1, 0],
    [0, 0, 0, 0, 0],
    [-1, 0, -1, 0, -1],
    [0, 0, 0, 1, 0]
])

# Hyperparameters for Q-learning
alpha = 0.1  # Learning rate (controls how much new information overrides old information)
gamma = 0.9  # Discount factor (weights the importance of future rewards)
epsilon = 0.8  # Exploration rate (probability of taking a random action)
epsilon_decay = 0.995  # Decay rate for epsilon (reduces exploration over time)
num_episodes = 500  # Number of training episodes (iterations)

# Initialize the Q-table with zeros
# Q-table dimensions: (rows, columns, number of actions)
# Each state (position) has a value for each action (up, down, left, right)
q_table = np.zeros((maze.shape[0], maze.shape[1], 4))

# Define possible actions with their corresponding changes in position
# 0: Up (-1 in row), 1: Down (+1 in row), 2: Left (-1 in column), 3: Right (+1 in column)
actions = {
    0: (-1, 0),  # Move up
    1: (1, 0),   # Move down
    2: (0, -1),  # Move left
    3: (0, 1)    # Move right
}

# Function to check if a position is valid (within bounds and not an obstacle)
def is_valid_position(position):
    x, y = position  # Unpack the coordinates
    # Check if position is within the maze boundaries and not an obstacle
    return 0 <= x < maze.shape[0] and 0 <= y < maze.shape[1] and maze[x, y] != -1

# Function to get the next valid position based on the current position and chosen action
def get_next_position(position, action):
    x, y = position  # Current position
    dx, dy = actions[action]  # Get changes in coordinates for the action
    next_position = (x + dx, y + dy)  # Calculate the new position
    # Return the new position if valid, otherwise stay in the current position
    return next_position if is_valid_position(next_position) else position

# Function to get the reward for the current position
def get_reward(position):
    # If the position is the goal, return a high reward
    if maze[position] == 1:
        return 10
    # If the position is an obstacle, return a penalty
    elif maze[position] == -1:
        return -1
    # Otherwise, return a small step penalty for moving
    return -0.1

# Main Q-learning loop for training the agent
for episode in range(num_episodes):
    position = (0, 0)  # Start at the top-left corner of the maze
    done = False  # Flag to indicate if the episode is finished

    while not done:
        # Epsilon-greedy policy: choose to explore or exploit
        if random.uniform(0, 1) < epsilon:
            action = random.randint(0, 3)  # Explore: select a random action
        else:
            action = np.argmax(q_table[position[0], position[1]])  # Exploit: select the action with the highest Q-value

        # Get the next position and the reward for that action
        next_position = get_next_position(position, action)
        reward = get_reward(next_position)

        # Get the best action for the next state based on the Q-table
        best_next_action = np.argmax(q_table[next_position[0], next_position[1]])

        # Update Q-value using the Bellman equation:
        # Q(state, action) = Q(state, action) + alpha * (reward + gamma * max(Q(next_state, next_action)) - Q(state, action))
        q_table[position[0], position[1], action] += alpha * (
            reward + gamma * q_table[next_position[0], next_position[1], best_next_action] -
            q_table[position[0], position[1], action]
        )

        # Move to the next position
        position = next_position

        # Check if the goal is reached; end the episode if so
        if maze[position] == 1:
            done = True

    # Decay the epsilon value after each episode to reduce the exploration rate
    epsilon *= epsilon_decay

# Print the Q-table after training is completed
print("Training completed!")
print(q_table)


Training completed!
[[[ 4.20039616  3.08572369  4.17092496  4.845851  ]
  [ 4.69778982  4.78138718  3.95270031  5.49539   ]
  [ 5.37941298  6.2171      4.70804609  5.31170782]
  [ 0.          0.          0.          0.        ]
  [-0.07175904 -0.04390937 -0.08410269 -0.06593549]]

 [[ 0.64688468  4.55416453  0.59882012  1.03532462]
  [ 0.          0.          0.          0.        ]
  [ 5.46014339  7.019       5.96615107  6.18286325]
  [ 0.          0.          0.          0.        ]
  [-0.07365385  2.33158083 -0.08785025  0.21729709]]

 [[ 0.82808929  1.23183802  1.65693154  5.9092048 ]
  [ 3.33913415  1.51737285  1.88361154  7.01109925]
  [ 6.12613804  6.94140551  6.05658201  7.91      ]
  [ 7.86909925  8.9         6.9441895   6.6153003 ]
  [ 0.37006488  2.16797956  7.83114352  2.83720338]]

 [[ 0.          0.          0.          0.        ]
  [ 0.72143871  3.27339778  0.35479833  0.14015308]
  [ 0.          0.          0.          0.        ]
  [ 7.81454546 10.          8.60632849